## Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MinMaxScaler
from surprise import SVD
from surprise.model_selection import cross_validate

In [2]:
df = pd.read_csv(r'C:\Users\Shibbs\Desktop\Praxis\CAPP\cwd\datasets\mappedknn_0.csv')

In [3]:
df.isnull().sum()

Unnamed: 0            0
event_time            0
event_type            0
product_id            0
category_id           0
price                 0
user_id               0
user_session        165
brand                 0
category_code         0
category_code_L1      0
category_code_L2      0
dtype: int64

## Model-based collaborative filtering system

 Recommend items to users based on purchase history and similarity of ratings provided by other users who bought items to that of a particular customer.

A model based collaborative filtering technique is closen here as it helps in making predictinfg products for a particular user by identifying patterns based on preferences from multiple user data.

### Utility Matrix based on products sold and user reviews

Utility Matrix : A utlity matrix consists of all possible user-item preferences (ratings) details represented as a matrix. The utility matrix is sparce as none of the users would buy all the items in the list, hence, most of the values are unknown.

In [4]:
df['user_score'] = df['event_type'].map({'view':1,'cart':10,'purchase':50})
df['user_purchase'] = df['event_type'].apply(lambda x: 1 if x=='purchase' else 0)
df['price_category'] = 0

# divide price range into 5 categories (Low to High) for each category code.
for i in df['category_code'].unique():
    # find all indices of given unique category_code and fill the price_category column of respective indices with 5 category range of price.
    df.loc[df['category_code']==i,'price_category'] = pd.qcut(x=df['price'][df['category_code']==i],q=5,labels=False, duplicates = 'drop')

In [5]:
df=df.dropna()

In [6]:
df.price_category=df.price_category+1

In [7]:
df.price_category.unique()

array([2., 3., 1., 4., 5.])

In [10]:
df

,Unnamed: 0,event_time,event_type,product_id,category_id,price,user_id,user_session,brand,category_code,category_code_L1,category_code_L2,user_score,user_purchase,price_category
0,0,2020-11-20 15:19:47 UTC,view,1642613,2.144416e+18,25.56,1.515916e+18,9cdf5Yzi4W,falcon,auto.accessories.immobilizer,auto,appliances,1,0,2.0
1,1,2020-10-18 18:29:34 UTC,view,1642613,2.144416e+18,25.56,1.515916e+18,DomDtlfN2x,falcon,auto.accessories.immobilizer,auto,appliances,1,0,2.0
2,2,2020-11-20 19:18:47 UTC,view,1642613,2.144416e+18,25.56,1.515916e+18,N5U9nr75XP,falcon,auto.accessories.immobilizer,auto,appliances,1,0,2.0
3,3,2020-12-20 01:25:28 UTC,view,1642613,2.144416e+18,25.56,1.515916e+18,apG36IgOcf,falcon,auto.accessories.immobilizer,auto,appliances,1,0,2.0
4,4,2021-02-24 15:59:50 UTC,view,1642613,2.144416e+18,25.56,1.515916e+18,7CZ3WuPoU3,falcon,auto.accessories.immobilizer,auto,appliances,1,0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885119,885119,2020-12-02 14:01:32 UTC,view,1529443,2.144420e+18,13.24,1.515920e+18,D2iN8WaxdK,wolta,construction.tools.light,construction,toys,1,0,1.0
885120,885120,2021-01-06 13:14:08 UTC,view,1529453,2.144420e+18,66.19,1.515920e+18,IoOI7VXzFo,wolta,construction.tools.light,construction,toys,1,0,5.0
885121,885121,2020-10-08 12:10:00 UTC,view,864264,2.144420e+18,20.63,1.515920e+18,kQHdSa7WoM,wuw,electronics.telephone.cover,electronics,toolkit,1,0,2.0
885122,885122,2020-10-08 12:10:02 UTC,view,864264,2.144420e+18,20.63,1.515920e+18,GAylf4yJHR,wuw,electronics.telephone.cover,electronics,toolkit,1,0,2.0


In [12]:
# new dataframe grouped by user_id and product_id, for each product id sum of all user scores and user purchases.
group = df.groupby(['user_id','product_id'])['user_score','user_purchase'].sum().reset_index()


group['user_purchase'] = group['user_purchase'].apply(lambda x: 1 if x>1 else x)
group['user_score'] = group['user_score'].apply(lambda x: 100 if x>100 else x)

#minmax scaler (1,5) applied on user score to get interaction score.
# std = MinMaxScaler(feature_range=(1,5))
# std.fit(group['user_score'].values.reshape(-1,1))
# group['interaction_score'] = std.transform(group['user_score'].values.reshape(-1,1))

# group = group.merge(df[['product_id','category_code','brand','price','price_category']].drop_duplicates('product_id'),on=['product_id'])

C:\Users\Shibbs\AppData\Local\Temp/ipykernel_13812/3593986703.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  group = df.groupby(['user_id','product_id'])['user_score','user_purchase'].sum().reset_index()


In [13]:
group['i_s'] =  group['user_score'] * df['price_category']

In [14]:
group = group.dropna()

In [15]:
group.head(50)

,user_id,product_id,user_score,user_purchase,i_s
0,1.515916e+18,105,1,0,2.0
1,1.515916e+18,73593,8,0,16.0
2,1.515916e+18,124883,1,0,2.0
3,1.515916e+18,125325,1,0,2.0
4,1.515916e+18,254763,4,0,8.0
5,1.515916e+18,461533,1,0,2.0
6,1.515916e+18,760044,1,0,2.0
7,1.515916e+18,886608,1,0,2.0
8,1.515916e+18,1038457,1,0,2.0
9,1.515916e+18,1399881,1,0,2.0


## sklearn surprise for building recommendation system

In [23]:
import pandas as pd
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV

In [24]:
df = group[['user_id', 'product_id', 'interaction_score']]

In [30]:
df['interaction_score'].std()

0.6542762078722747

In [27]:
reader = Reader(rating_scale=(1.0,5.0))
data = Dataset.load_from_df(df, reader)
benchmark = []
# Iterate over all algorithms
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), 
                  KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), 
                  BaselineOnly()]:
    
 # Perform cross validation
 results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
 
 # Get results & append algorithm name
 tmp = pd.DataFrame.from_dict(results).mean(axis=0)
 tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
 benchmark.append(tmp)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...


In [28]:
surprise_results = pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')

In [29]:
surprise_results

,test_rmse,fit_time,test_time
Algorithm,,,
BaselineOnly,0.634850,1.095021,1.357112
SVD,0.645739,16.300415,1.447399
SVDpp,0.648014,269.375422,10.414839
KNNBaseline,0.659339,7.771721,21.019700
KNNBasic,0.661188,6.673701,19.228469
SlopeOne,0.664549,119.485142,31.012737
KNNWithMeans,0.664909,6.826701,19.948898
KNNWithZScore,0.671623,7.309101,20.717885
NMF,0.671889,24.640808,1.400577


In [33]:
print('Using ALS')
bsl_options = {'method': 'als',
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }
algo = BaselineOnly(bsl_options=bsl_options)
cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)

Using ALS
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


{'test_rmse': array([0.15395466, 0.15458058, 0.15622601]),
 'fit_time': (0.6345880031585693, 0.702836275100708, 0.7109251022338867),
 'test_time': (0.839890718460083, 1.0034582614898682, 1.0279719829559326)}

In [34]:
trainset, testset = train_test_split(data, test_size=0.25)
algo = BaselineOnly(bsl_options=bsl_options)
predictions = algo.fit(trainset).test(testset)
accuracy.rmse(predictions)

Estimating biases using als...
RMSE: 0.1551


0.15506718206017595

### SVD Recommendation Engine

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(group.drop('i_s', axis = 1), 
                                                    group['i_s'],
                                                    test_size = 0.3, 
                                                   )

In [19]:
X_train_matrix = pd.pivot_table(X_train,values='user_score',index='user_id',columns='product_id')
X_train_matrix = X_train_matrix.fillna(0)

In [20]:
X_train_matrix.shape

(12044, 46385)

In [21]:
X = X_train_matrix.T
X.head()

user_id,1.515916e+18,1.515916e+18,1.515916e+18,1.515916e+18,1.515916e+18,1.515916e+18,1.515916e+18,1.515916e+18,1.515916e+18,1.515916e+18,...,1.515916e+18,1.515916e+18,1.515916e+18,1.515916e+18,1.515916e+18,1.515916e+18,1.515916e+18,1.515916e+18,1.515916e+18,1.515920e+18
product_id,,,,,,,,,,,,,,,,,,,,,
102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
105,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
526,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
561,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
X.shape

(46385, 12044)

In [23]:
X1 = X

In [24]:
SVD = TruncatedSVD(n_components=10)
decomposed_matrix = SVD.fit_transform(X)
decomposed_matrix.shape

C:\Users\Shibbs\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float']. An error will be raised in 1.2.
  warnings.warn(


(46385, 10)

In [ ]:
correlation_matrix = np.corrcoef(decomposed_matrix)
correlation_matrix.shape

### Save correlation matrix

In [23]:
import numpy as np
from pathlib import Path

In [24]:
path = Path(r'C:\Users/Shibbs/Desktop/Praxis/CAPP/cwd/datasets/').expanduser()
path.mkdir(parents=True, exist_ok=True)

In [25]:
np.save(path/'correlation_matrix', correlation_matrix)

In [6]:
correlation_matrix_loaded = np.load(r'C:\Users/Shibbs/Desktop/Praxis/CAPP/cwd/datasets/correlation_matrix.npy')

## Isolating Product ID # 6117036094 from the Correlation Matrix
 Assuming the customer buys Product ID # 6117036094 (randomly chosen)

In [ ]:
X.index[344]

Index # of product ID purchased by customer



In [95]:
i = 1795543

product_names = list(X.index)
product_ID = product_names.index(i)
print(product_ID)
df.category_code[product_ID]

34544


'computers.components.videocards'

Correlation for all items with the item purchased by this customer based on items rated by other customers people who bought the same product

In [96]:
correlation_product_ID = correlation_matrix[product_ID]
correlation_product_ID.shape

(46508,)

Recommending top 10 highly correlated products in sequence


Product Id # Here are the top 10 products to be displayed by the recommendation system to the above customer based on the purchase history of other customers in the website.

In [111]:
Recommend = list(X.index[correlation_product_ID > 0.96])

# Removes the item already bought by the customer
Recommend.remove(i) 
df.category_code[Recommend[0:9]]

138460                 computers.components.cpu
254139    electronics.components.metal_packages
287402               electronics.audio.acoustic
287787               computers.peripherals.wifi
573363          computers.components.videocards
598848            computers.peripherals.printer
640366          computers.components.videocards
673282                    electronics.telephone
781858             computers.peripherals.camera
Name: category_code, dtype: object

In [10]:
group = df.groupby(['user_id','product_id'])['user_score','user_purchase'].sum().reset_index()

C:\Users\Shibbs\AppData\Local\Temp/ipykernel_11516/57596884.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  group = df.groupby(['user_id','product_id'])['user_score','user_purchase'].sum().reset_index()


In [20]:
group

,user_id,product_id,user_score,user_purchase
0,1.515916e+18,105,1,0
1,1.515916e+18,73593,8,0
2,1.515916e+18,124883,1,0
3,1.515916e+18,125325,1,0
4,1.515916e+18,254763,4,0
...,...,...,...,...
511776,1.515920e+18,4183778,2,0
511777,1.515920e+18,4183779,1,0
511778,1.515920e+18,4183781,7,0
511779,1.515920e+18,4183819,1,0
